In [ ]:
import json
with open("cache/accuracies_sorted.json", "r") as file:
    accuracies_sorted = json.load(file)

import matplotlib.pyplot as plt
from collections import defaultdict
i = 0
accs_by_layer = defaultdict(list)
for layer in range(len(hooked_model.model.model.layers)):
    for head in range(hooked_model.model.model.layers[0].self_attn.num_heads):
        accs_by_layer[layer].append(accuracies[i])
        i += 1
        
fig, axs = plt.subplots(8, 4, sharey="row")
fig.set_size_inches((15, 15))
for i, layer in enumerate(accs_by_layer):
    y = i // 4
    x = i % 4
    axs[y][x].plot(accs_by_layer[layer])
    axs[y][x].set_title(f"probing accuracy, layer {layer}")

for i in range(8):
    for j in range(4):
        axs[i][j].set_xticks(np.arange(len(accs_by_layer[0])), labels=np.arange(len(accs_by_layer[0])))
        if i == 1:
            axs[i][j].set_xlabel("head")
fig.tight_layout()
plt.savefig("images/probing_accuracies.png")

In [ ]:
import numpy as np
with open("cache/accs_by_k_alpha_on_probing_data.json", "r") as file:
    accs_by_k_alpha_json = json.load(file)
accs_by_k_alpha = {}
for k, v in accs_by_k_alpha_json.items():
    accs_by_k_alpha[eval(k)] = eval(v)

ks = sorted(list(set([k for k, a in accs_by_k_alpha.keys()])))

with open("cache/accs_by_k_alpha_on_probing_data_no_invervention.json", "r") as file:
    accs_no_intervention_json = json.load(file)
for k in ks:
    accs_by_k_alpha[(k, 0)] = eval(accs_no_intervention_json)


alphas = sorted(list(set([a for k, a in accs_by_k_alpha.keys()])))

a = np.zeros((len(ks), len(alphas)))
for k_i, k in enumerate(ks):
    for a_i, alpha in enumerate(alphas):
        a[k_i, a_i] = f"{np.mean(accs_by_k_alpha[(k, alpha)]).item():.3f}"


import matplotlib.pyplot as plt
fig, ax = plt.subplots()
fig.set_size_inches((4, 4))
im = ax.imshow(a, cmap="PuBu")

ax.set_xticks(np.arange(len(alphas)), labels=alphas)
ax.set_yticks(np.arange(len(ks)), labels=ks)

for i in range(len(ks)):
    for j in range(len(alphas)):
        text = ax.text(j, i, a[i, j],
                       ha="center", va="center", color="black")

ax.set_title("MC accuracy")
ax.set_xlabel("alpha")
ax.set_ylabel("K")
fig.tight_layout()
plt.show()
plt.savefig("images/MC_acc_steering.png")